In [0]:
from keras.layers import Dense, Dropout, Flatten, Input, ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras import regularizers
from keras.models import Model
from keras import optimizers
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from cv2 import cv2
from google.colab import drive
import pandas as pd 
import keras 
from keras.models import load_model
import os
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.models import Sequential
from keras.optimizers import RMSprop, SGD, Adam
import gc

In [2]:
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/APM'  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd /content/gdrive/My Drive/APM/model

/content/gdrive/My Drive/APM/model


In [4]:
model = load_model('model_v6_23.hdf5')












Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [37]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 8, 8, 128)         147584    
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 4, 4, 128)       

In [5]:
model = Model(inputs=model.inputs, outputs=model.layers[-5].output)
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54_input (InputLayer) (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 46, 46, 32)        320       
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 44, 44, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 8, 8, 128)         147

In [6]:
new_model = Sequential()
new_model.add(model)
new_model.summary()

new_model.add(Conv2D(8, kernel_size=(1, 1), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
# # model.add(BatchNormalization())

new_model.add(Conv2D(8, kernel_size=(4, 4), activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
# model.add(BatchNormalization())

new_model.add(Flatten())

new_model.add(Activation("softmax"))
new_model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 4, 4, 128)         240256    
Total params: 240,256
Trainable params: 240,256
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 4, 4, 128)         240256    
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 8)           1032      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 8)           1032      
_________________________________________________________________
flatten_1 (Flatten)          (None, 8)                 0         
__________________________

In [0]:
new_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

In [8]:
cd '/content/gdrive/My Drive/APM/Dataset/unzipped/'

/content/gdrive/My Drive/APM/Dataset/unzipped


In [0]:
test_on = ['Actor_17', 'Actor_20']
def video_generator(skip):
  j = 1
  X=[]
  Y=[]
 
  for actor in os.listdir('./'):
  #for actor in [('Actor_01'),]:
    print(actor)

    if ('Actor' not in actor) or (actor in skip):
      
      continue # skipping the extra folders
    path = os.path.join('./',actor)
    
    # progress tracking
    l = len(os.listdir(path))
    i = 0
    
    #print('Training on',actor)
    j += 1

    for filename in os.listdir(path):
      
      print(filename)
      path_inner = os.path.join(path, filename)
      file_emotion = [0]*8
      file_emotion[int(path_inner[17:19])-1] = 1
      print(file_emotion)
      print(path_inner)

      cap = cv2.VideoCapture(path_inner)
      x = []
      y = []
      
     

      frame_count = 0
      while True:
        
        frame_count += 1
        # read the next frame from the file
        (grabbed, frame) = cap.read()

        # If the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
          
          #print('Done,',i,'frames captured.')
          break
        # only grab each 5th frame to capture a slightly different expression each time; makes computation 5x faster
        elif frame_count % 20 == 0:
          frame= cv2.resize(frame, (48,48))
          frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          frame = np.reshape(frame, [frame.shape[0], frame.shape[1], 1])
          x.append(frame)
          y.append(file_emotion)

      
      i += 1
      #print('    Training on',filename, int(i/l*100), '%')
      #cnn.train_on_batch(x,y)
      x = np.array(x)
      y = np.array(y)
      i += 1
      #print('    Training on',filename, int(i/l*100), '%')
      #cnn.train_on_batch(x,y)
      yield (x,y)
      del x
      del y
      
      
  

In [10]:
new_model.fit_generator(video_generator(test_on), steps_per_epoch=25,epochs = 50,verbose = 1)
new_model.save('transfer.hdf5')

Running

In [0]:
model=load_model('transfer.hdf5')

In [0]:
def testOnActor(actor):
  actor_dir = './'+actor+'/'
  loss = []
  acc = []
  #Test
  i = 0
  for filename in os.listdir(actor_dir):
    i += 1
    #if i > 10:
    #  break
   
    path = os.path.join(actor_dir, filename)
    file_emotion = [0]*8
    file_emotion[int(path[17:19])-1] = 1

    cap = cv2.VideoCapture(path)
    
    x = []
    y = []
    frame_count = 0
    while True:
      # read the next frame from the file
      (grabbed, frame) = cap.read()
      frame_count += 1

      # If the frame was not grabbed, then we have reached the end
      # of the stream
      if not grabbed:
        #print('Done,',i,'frames captured.')
        break
      
      elif frame_count % 10 == 0:
      
        frame=cv2.resize(frame, (48,48))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = np.reshape(frame, [frame.shape[0], frame.shape[1], 1])

        x.append(frame)
        y.append(file_emotion)
        

    x = np.array(x)
    y = np.array(y)
    
    
    #print('Testing on',filename)
    test = model.test_on_batch(x,y)

    

    loss.append(test[0])
    acc.append(test[1])
    del x
    del y   
    gc.collect()

    

  return loss,acc




In [37]:
loss23,acc23 = testOnActor('Actor_23')
print('Actor 23:')
print('Average Loss =',np.array(loss23).mean())
print('Average Accuracy =',np.array(acc23).mean())

Actor 23:
Average Loss = 13.976456
Average Accuracy = 0.13333334


In [38]:
loss24,acc24 = testOnActor('Actor_24')
print('Actor 24:')
print('Average Loss =',np.array(loss24).mean())
print('Average Accuracy =',np.array(acc24).mean())


Actor 24:
Average Loss = 13.976457
Average Accuracy = 0.13333334
